## Install Dependencies

In [ ]:
!pip install \
  langchain \
  langchain-community \
  langchain-chroma \
  langchain-google-genai \
  chromadb \
  pypdf \
  python-dotenv \
  tqdm


## Get Gemini API Working

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Load PDFs

In [2]:
from langchain_community.document_loaders import PyPDFLoader
import os

PDF_DIR = "data/event_pdfs"

def load_pdfs(pdf_dir):
    documents = []
    for file in os.listdir(pdf_dir):
        if file.endswith(".pdf"):
            path = os.path.join(pdf_dir, file)
            loader = PyPDFLoader(path)
            docs = loader.load()

            # Attach paper ID to metadata
            for d in docs:
                d.metadata["paper_id"] = file.replace(".pdf", "")
            documents.extend(docs)

    return documents


raw_docs = load_pdfs(PDF_DIR)
print(f"Loaded {len(raw_docs)} pages")


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 781 0 (offset 0)
Ignoring wrong pointing object 785 0 (offset 0)
Ignoring wrong pointing object 1369 0 (offset 0)


Loaded 467 pages


In [3]:
## load pdfs

from langchain_community.document_loaders import PyPDFLoader
import os
from tqdm import tqdm

data_dir = "data/event_pdfs"

documents = []
for file in tqdm(os.listdir(data_dir)):
    if file.endswith(".pdf"):
        path = os.path.join(data_dir, file)

        # loader: recieves path of a single file, returns list of Document objects  >> docs is a list
        # Each Document object represents a page from the PDF
        loader = PyPDFLoader(path)
        docs = loader.load()

        # Attach paper ID to metadata
        for d in docs:
            d.metadata["paper_id"] = file.replace(".pdf", "")

        # use extend to add multiple items to a list, not append to create nested list
        documents.extend(docs)

print(f"Loaded {len(documents)} pages")

 23%|██▎       | 7/31 [00:02<00:07,  3.03it/s]Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 781 0 (offset 0)
Ignoring wrong pointing object 785 0 (offset 0)
Ignoring wrong pointing object 1369 0 (offset 0)
100%|██████████| 31/31 [00:30<00:00,  1.03it/s]

Loaded 467 pages


In [4]:
# Split into Chunks  

In [5]:
# ===============================================================================================================
#                                               Documents into Chunk
# ===============================================================================================================

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

print(f"\nnum. chunks {len(chunks)}")


num. chunks 6025


### Explanation (don't run)

In [ ]:
# What page_content might look like for page 1 of a research paper:
docs[0].page_content = """
Event Cameras: A New Paradigm for Computer Vision
John Doe, Jane Smith
University of Technology

Abstract—Event cameras are bio-inspired sensors...

I. INTRODUCTION
Traditional frame-based cameras...

Figure 1: Comparison of event camera vs conventional camera.
"""
# ===========================================================================

# Before your modification:
docs[0].metadata = {
    "source": "data/event_pdfs/event_camera_survey.pdf",  # File path
    "page": 0  # Page number (0-indexed, so page 0 = actual page 1)
}

# After your modification with paper_id:
docs[0].metadata = {
    "source": "data/event_pdfs/event_camera_survey.pdf",
    "page": 0,
    "paper_id": "event_camera_survey"  # ← Your added field!
}

# ===========================================================================

# documents list would contain 15 Document objects (5+3+7)
len(documents)  # Returns: 15

# Accessing specific documents:
documents[0]   # Page 1 of event_camera_basics.pdf
documents[4]   # Page 5 of event_camera_basics.pdf (last page of this PDF)
documents[5]   # Page 1 of davis_sensors.pdf
documents[7]   # Page 3 of davis_sensors.pdf (last page)
documents[8]   # Page 1 of applications_robotics.pdf
documents[14]  # Page 7 of applications_robotics.pdf (last page)

## Create Embeddings + ChromaDB

In [9]:
# embedding model
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")


# Store in ChromaDB
from langchain_chroma import Chroma

DB_NAME = "./vector_db_event_cameras"  # Name of the ChromaDB directory

if os.path.exists(DB_NAME):
        Chroma(persist_directory=DB_NAME, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents( documents=chunks, embedding=embeddings, persist_directory=DB_NAME )


# print info about the stored vectors, _collection is the internal Chroma object
collection = vectorstore._collection
num_embedding = collection.count()
print(f"\nnumber of embedding vectors {num_embedding}")

sample = collection.get(limit = 1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample)
print(f"dimensions : {dimensions}")



number of embedding vectors 6025
dimensions : 768


## Build the RAG Pipeline

In [24]:
## Load Vector Store
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama



DB_NAME =  "./vector_db_event_cameras"
embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma(persist_directory = DB_NAME, embedding_function=embeddings)     # path, Embedding Model


# when a query comes >> search and retrieve
retriever = vectorstore.as_retriever()


# define LLM model
llm = ChatOllama(model="phi",temperature=0) 

In [25]:
from langchain_core.prompts import PromptTemplate

SYSTEM_PROMPT = PromptTemplate(
    template="""
You are a research assistant specializing in event-based vision.

Use the following context from research papers to answer the question.
If the answer is not in the context, say you don't know.

Context:
{context}

Question:
{question}

Answer (technical, concise, cite papers if possible):
""",
    input_variables=["context", "question"]
)


In [26]:
from langchain_classic.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": SYSTEM_PROMPT}
)


In [27]:
query = "How do event cameras differ from frame-based cameras in SLAM applications?"

result = qa_chain(query)

print("Answer:\n", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(doc.metadata.get("source"))


Answer:
 
Event cameras are well suited for applications in which standard frame cameras are affected by motion blur, pixel saturation, and high latency. Despite the remarkable properties of event cameras, we are still at the dawn of event-based vision and their adoption in real systems is currently limited. This implies scarce availability of algorithms, datasets, and tools to manipulate and process events. Additionally, most of the available datasets have


Sources:
data/event_pdfs\NeurIPS-2020-learning-to-detect-objects-with-a-1-megapixel-event-camera-Paper.pdf
data/event_pdfs\Event-Based_Vision_A_Survey.pdf
data/event_pdfs\Event-Based_Vision_A_Survey.pdf
data/event_pdfs\Pan_Bringing_a_Blurry_Frame_Alive_at_High_Frame-Rate_With_an_CVPR_2019_paper.pdf


### Explain

In [ ]:
# RetrievalQA
# a pre-built chain in LangChain that combines: 1. Retrieval  2. Question Answering 

# chain_type="stuff"
# means: All documents concatenated into single prompt